In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
base_path = '../../'
sys.path.append(base_path)

In [ ]:
from cooldata.dgl_flow_field_dataset import DGLSurfaceFlowFieldDataset
from cooldata.pyvista_flow_field_dataset import PyvistaFlowFieldDataset
ds_pv = PyvistaFlowFieldDataset.load_from_huggingface("datasets/pyvista_s",num_samples=3)
ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface_s'),ds_pv)
#ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface'))
pos= ds_dgl[0].ndata['Position']
ds_dgl[0]

Loaded 3 samples from '/nfs/homedirs/peo/flow_field_dataset/examples/meshgraphnet/datasets/pyvista_s'.
Loaded 3 samples from 'datasets/pyvista_s'.


Converting Pyvista dataset to DGLGraphs (surface):   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
ds_pv[0].surface_data[0].combine()

UnstructuredGrid (0x7046bc7d1ea0)
  N Cells:    35771
  N Points:   37371
  X Bounds:   -4.337e-19, 5.000e-01
  Y Bounds:   0.000e+00, 1.000e-01
  Z Bounds:   0.000e+00, 2.000e-02
  N Arrays:   9

In [ ]:
edges_from, edges_to = find_k_nearest_neighbors(pos.to("cuda"),k=10)

In [ ]:
graph = dgl.graph((edges_from, edges_to), num_nodes=pos.shape[0]).to("cpu")
graph.ndata['Position'] = pos
for key in ds_dgl[0].ndata:
    graph.ndata[key] = ds_dgl[0].ndata[key]
graph.edata["dx"] = graph.ndata['Position'][graph.edges()[1]] - graph.ndata['Position'][graph.edges()[0]]

In [ ]:
graph

Graph(num_nodes=34829, num_edges=362546,
      ndata_schemes={'Position': Scheme(shape=(3,), dtype=torch.float32), 'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})

In [9]:
import pyvista as pv
pv.set_jupyter_backend('trame')
ds_dgl.plot_surface(ds_dgl[0],"Pressure")

Widget(value='<iframe src="http://localhost:42461/index.html?ui=P_0x751995137f10_1&reconnect=auto" class="pyvi…